In [1]:
import Trade
from ConnectDB import *
import datetime
import pytz
import pandas as pd

In [168]:
# example
AAL_long_order = Trade.go_long('AAL',100,100,120,80)

Paper API connected
Go long order submitted:
Order ID: 173b29dc-0871-4db5-b5fa-149505481c74
Entry point: 100
Take profit point: 120
Stop loss point: 79.2


In [169]:
# cancel all orders
Trade.api_access('paper').cancel_all_orders()

Paper API connected


In [2]:
# Check database connection,if not connected, connect it.
try:
    con
except NameError:
    # Connect to database
    con,cur = connect_data_base()
    print('Initial connection to PostgreSQL DataBase')
else:
    if con.closed !=0:
    # Connect to database
        con,cur = connect_data_base()
        print('Reconnected to PostgreSQL DataBase')
    print(con.closed)
        

DataBase Connected!
Version: PostgreSQL 14.5 (Homebrew) on aarch64-apple-darwin22.1.0, compiled by Apple clang version 14.0.0 (clang-1400.0.29.202), 64-bit
Initial connection to PostgreSQL DataBase


In [12]:
# Connect to database
con,cur = connect_data_base()

DataBase Connected!
Version: PostgreSQL 14.5 (Homebrew) on aarch64-apple-darwin22.1.0, compiled by Apple clang version 14.0.0 (clang-1400.0.29.202), 64-bit


In [15]:
# Fuction for acquiring data from database.
def get_data(data_type = 'stock_bar'):
    # acquire wall street date
    def get_wall_street_date():
        """
        return wall street date
        """
        tz = pytz.timezone('America/New_York')
        return datetime.datetime.now(tz).strftime('%Y-%m-%d')

    # acquire latest data from realtime database (last 60 minites)
    wall_street_date = get_wall_street_date()
    table_name = "_" + wall_street_date.replace('-','_') + f'_{data_type}'
    cur.execute("SELECT * FROM {} ORDER BY time DESC LIMIT 60".format(table_name))
    data = cur.fetchall()

    # Acquire table columns names
    cur.execute("SELECT * FROM information_schema.columns WHERE table_schema = 'public' AND table_name   = '{}'".format(table_name))
    rows = cur.fetchall()
    table_columns = []
    for row in rows:
        table_columns.append(row[3])


    data_df = pd.DataFrame(data,columns= table_columns)
    return data_df

In [16]:
# call the get_data function and show the database.
data_df = get_data(data_type = 'crypto_bar')
data_df.head()

,symbol,time,type,close,high,low,open,trade,volume,vwap
0,BTC/USD,2022-11-12 22:37:00-05:00,b,16883.69,16883.69,16870.6,16877.58,18,0.674798,16873.4968092525
1,BTC/USD,2022-11-12 22:36:00-05:00,b,16875.24,16887.49,16868.24,16872.61,57,3.250854,16877.6932017618
2,BTC/USD,2022-11-12 22:35:00-05:00,b,16869.02,16869.32,16855.5,16860.4,26,1.396087,16856.7092792283
3,BTC/USD,2022-11-12 22:34:00-05:00,b,16860.41,16867.21,16855.51,16857.48,72,1.967726,16861.0785133499
4,BTC/USD,2022-11-12 22:33:00-05:00,b,16857.48,16860.82,16850,16850,62,0.581412,16858.0173809278
